# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

  Cloning git://github.com/Theano/Theano.git to /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-req-build-8g5oafk2
  Running command git clone -q git://github.com/Theano/Theano.git /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-req-build-8g5oafk2
  Created wheel for Theano: filename=Theano-1.0.4+21.g8f510a1f0-cp36-none-any.whl size=2666379 sha256=041cb3a9e761cab5d820669c49be2cb71215f819b38d3ca8810aee93f4551093
  Stored in directory: /private/var/folders/xv/6xdsnhm10jg5y1g0bm3kp9140000gp/T/pip-ephem-wheel-cache-pri53nb7/wheels/ae/32/7c/62beb8371953eb20c271b3bac7d0e56e1a2020d46994346b52
Successfully built Theano
  Found existing installation: Theano 1.0.4+21.g8f510a1f0
    Uninstalling Theano-1.0.4+21.g8f510a1f0:
      Successfully uninstalled Theano-1.0.4+21.g8f510a1f0
  Using cached https://files.pythonhosted.org/packages/27/8c/7608ba709bd536bc2bccb0d1abbb70aafe9cf7e0170353b4b720ed54cb71/tensorflow-1.14.0-cp36-cp36m-macosx_10_11_x86_64.whl
  Using cached http

# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

/home/icaro/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/home/icaro/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/icaro/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [8]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 199us/step - loss: 0.4887 - accuracy: 0.7959
Epoch 2/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.4313 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4278 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4242 - accuracy: 0.7985
Epoch 5/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.4205 - accuracy: 0.8163
Epoch 6/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.4176 - accuracy: 0.8232
Epoch 7/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.4154 - accuracy: 0.8282
Epoch 8/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4136 - accuracy: 0.8305
Epoch 9/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.4122 - accuracy: 0.8317
Epoch 10/100
8000/8000 [==============================] - 1s 8

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [9]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1532   63]
 [ 253  152]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [11]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 107us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 98us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 100us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 93us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 93us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 86us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 89us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 91us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 89us/step - loss: 3.1467 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 1s 8

In [12]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


# Initialising the 3rd ANN

In [14]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.6162 - accuracy: 0.7940
Epoch 2/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.5350 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.5122 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.5071 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 86us/step - loss: 0.5061 - accuracy: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 85us/step - loss: 0.5060 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.5059 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================] - 1s 87u

In [15]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1595    0]
 [ 405    0]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

[0.83125001 0.83625001 0.83249998 0.82375002 0.85374999 0.82999998
 0.83125001 0.84125    0.82375002 0.83749998]
0.8341250002384186
0.008386179405193228


In [14]:
print(accuracies)
print(accuracies.mean())
print(accuracies.std())

[0.83125001 0.83625001 0.83249998 0.82375002 0.85374999 0.82999998
 0.83125001 0.84125    0.82375002 0.83749998]
0.8341250002384186
0.008386179405193228


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [18]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [19]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [10, 24, 32],
              'epochs': [50, 100, 200],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/50
7200/7200 [==============================] - 1s 129us/step - loss: 0.4887 - accuracy: 0.7967
Epoch 2/50
7200/7200 [==============================] - 1s 105us/step - loss: 0.4357 - accuracy: 0.7971
Epoch 3/50
7200/7200 [==============================] - 1s 109us/step - loss: 0.4300 - accuracy: 0.7971
Epoch 4/50
7200/7200 [==============================] - 1s 107us/step - loss: 0.4267 - accuracy: 0.7971
Epoch 5/50
7200/7200 [==============================] - 1s 116us/step - loss: 0.4281 - accuracy: 0.8032
Epoch 6/50
7200/7200 [==============================] - 1s 116us/step - loss: 0.4276 - accuracy: 0.8214
Epoch 7/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4261 - accuracy: 0.8238
Epoch 8/50
7200/7200 [==============================] - 1s 114us/step - loss: 0.4268 - accuracy: 0.8254
Epoch 9/50
7200/7200 [==============================] - 1s 108us/step - loss: 0.4236 - accuracy: 0.8251
Epoch 10/50
7200/7200 [==============================] - 1s 118u

7200/7200 [==============================] - 1s 103us/step - loss: 0.4319 - accuracy: 0.7975
Epoch 8/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.4310 - accuracy: 0.7975
Epoch 9/50
7200/7200 [==============================] - 1s 99us/step - loss: 0.4319 - accuracy: 0.7975
Epoch 10/50
7200/7200 [==============================] - 1s 100us/step - loss: 0.4289 - accuracy: 0.7975
Epoch 11/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4297 - accuracy: 0.7975
Epoch 12/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4293 - accuracy: 0.7975
Epoch 13/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4284 - accuracy: 0.7975
Epoch 14/50
7200/7200 [==============================] - 1s 97us/step - loss: 0.4298 - accuracy: 0.7975
Epoch 15/50
7200/7200 [==============================] - 1s 99us/step - loss: 0.4280 - accuracy: 0.7975
Epoch 16/50
7200/7200 [==============================] - 1s 101us/step - 

7200/7200 [==============================] - 1s 96us/step - loss: 0.4293 - accuracy: 0.8196
Epoch 14/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.4299 - accuracy: 0.8229
Epoch 15/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.4287 - accuracy: 0.8250
Epoch 16/50
7200/7200 [==============================] - 1s 96us/step - loss: 0.4278 - accuracy: 0.8221
Epoch 17/50
7200/7200 [==============================] - 1s 97us/step - loss: 0.4274 - accuracy: 0.8233
Epoch 18/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.4277 - accuracy: 0.8231
Epoch 19/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.4274 - accuracy: 0.8249
Epoch 20/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4279 - accuracy: 0.8281
Epoch 21/50
7200/7200 [==============================] - 1s 107us/step - loss: 0.4270 - accuracy: 0.8267
Epoch 22/50
7200/7200 [==============================] - 1s 97us/step - lo

7200/7200 [==============================] - 1s 94us/step - loss: 0.3728 - accuracy: 0.8433
Epoch 20/50
7200/7200 [==============================] - 1s 91us/step - loss: 0.3692 - accuracy: 0.8449
Epoch 21/50
7200/7200 [==============================] - 1s 94us/step - loss: 0.3638 - accuracy: 0.8501
Epoch 22/50
7200/7200 [==============================] - 1s 95us/step - loss: 0.3725 - accuracy: 0.8438
Epoch 23/50
7200/7200 [==============================] - 1s 97us/step - loss: 0.3747 - accuracy: 0.8451
Epoch 24/50
7200/7200 [==============================] - 1s 98us/step - loss: 0.3669 - accuracy: 0.8485
Epoch 25/50
7200/7200 [==============================] - 1s 90us/step - loss: 0.3692 - accuracy: 0.8439
Epoch 26/50
7200/7200 [==============================] - 1s 90us/step - loss: 0.3660 - accuracy: 0.8482
Epoch 27/50
7200/7200 [==============================] - 1s 97us/step - loss: 0.3657 - accuracy: 0.8496
Epoch 28/50
7200/7200 [==============================] - 1s 90us/step - loss

7200/7200 [==============================] - 1s 113us/step - loss: 0.4156 - accuracy: 0.8274
Epoch 26/50
7200/7200 [==============================] - 1s 104us/step - loss: 0.4093 - accuracy: 0.8340
Epoch 27/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4124 - accuracy: 0.8328
Epoch 28/50
7200/7200 [==============================] - 1s 105us/step - loss: 0.4124 - accuracy: 0.8335
Epoch 29/50
7200/7200 [==============================] - 1s 105us/step - loss: 0.4153 - accuracy: 0.8292
Epoch 30/50
7200/7200 [==============================] - 1s 103us/step - loss: 0.4131 - accuracy: 0.8322
Epoch 31/50
7200/7200 [==============================] - 1s 101us/step - loss: 0.4127 - accuracy: 0.8313
Epoch 32/50
7200/7200 [==============================] - 1s 102us/step - loss: 0.4104 - accuracy: 0.8353
Epoch 33/50
7200/7200 [==============================] - 1s 97us/step - loss: 0.4127 - accuracy: 0.8304
Epoch 34/50
7200/7200 [==============================] - 1s 96us/ste

...

7200/7200 [==============================] - 0s 39us/step - loss: 0.4230 - accuracy: 0.8313
Epoch 71/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4228 - accuracy: 0.8336
Epoch 72/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4226 - accuracy: 0.8343
Epoch 73/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4210 - accuracy: 0.8333
Epoch 74/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4221 - accuracy: 0.8315
Epoch 75/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4223 - accuracy: 0.8318
Epoch 76/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4219 - accuracy: 0.8335
Epoch 77/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4204 - accuracy: 0.8340
Epoch 78/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4221 - accuracy: 0.8328
Epoch 79/100
7200/7200 [==============================] - 0s 40us/st

7200/7200 [==============================] - 0s 38us/step - loss: 0.4258 - accuracy: 0.8269
Epoch 26/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.4299 - accuracy: 0.8247
Epoch 27/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.4286 - accuracy: 0.8250
Epoch 28/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4239 - accuracy: 0.8244
Epoch 29/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.4262 - accuracy: 0.8275
Epoch 30/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.4257 - accuracy: 0.8274
Epoch 31/100
7200/7200 [==============================] - 0s 36us/step - loss: 0.4253 - accuracy: 0.8299
Epoch 32/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4239 - accuracy: 0.8268
Epoch 33/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.4284 - accuracy: 0.8283
Epoch 34/100
7200/7200 [==============================] - 0s 40us/st

7200/7200 [==============================] - 0s 40us/step - loss: 0.3951 - accuracy: 0.8400
Epoch 81/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.3973 - accuracy: 0.8343
Epoch 82/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3965 - accuracy: 0.8393
Epoch 83/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.3996 - accuracy: 0.8344
Epoch 84/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3974 - accuracy: 0.8371
Epoch 85/100
7200/7200 [==============================] - 0s 38us/step - loss: 0.3968 - accuracy: 0.8394
Epoch 86/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3922 - accuracy: 0.8417
Epoch 87/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.3996 - accuracy: 0.8313
Epoch 88/100
7200/7200 [==============================] - 0s 39us/step - loss: 0.3980 - accuracy: 0.8364
Epoch 89/100
7200/7200 [==============================] - 0s 40us/st

7200/7200 [==============================] - 0s 42us/step - loss: 0.4232 - accuracy: 0.8319
Epoch 36/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.4236 - accuracy: 0.8326
Epoch 37/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.4228 - accuracy: 0.8301
Epoch 38/100
7200/7200 [==============================] - 0s 43us/step - loss: 0.4265 - accuracy: 0.8313
Epoch 39/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.4222 - accuracy: 0.8306
Epoch 40/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.4257 - accuracy: 0.8315
Epoch 41/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.4239 - accuracy: 0.8313
Epoch 42/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.4235 - accuracy: 0.8300
Epoch 43/100
7200/7200 [==============================] - 0s 44us/step - loss: 0.4258 - accuracy: 0.8297
Epoch 44/100
7200/7200 [==============================] - 0s 42us/st

7200/7200 [==============================] - 0s 42us/step - loss: 0.3809 - accuracy: 0.8419
Epoch 92/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3841 - accuracy: 0.8407
Epoch 93/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3855 - accuracy: 0.8399
Epoch 94/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3821 - accuracy: 0.8403
Epoch 95/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3820 - accuracy: 0.8396
Epoch 96/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3796 - accuracy: 0.8439
Epoch 97/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3797 - accuracy: 0.8404
Epoch 98/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3828 - accuracy: 0.8406
Epoch 99/100
7200/7200 [==============================] - 0s 40us/step - loss: 0.3845 - accuracy: 0.8413
Epoch 100/100
7200/7200 [==============================] - 0s 42us/s

In [10]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(accuracies.mean())
print(accuracies.std())

0.8360000014305115
0.010559358530310566


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)